In [ ]:
from google.colab import drive

drive.mount('/content/drive')


In [ ]:
import sys
import random
import numpy as np
import matplotlib.pyplot as plt
import time
import pathlib
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import tensorflow as tf 
import os 
import matplotlib.image as mpimg
import random
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from sklearn.model_selection import train_test_split 
from tensorflow.keras.utils import load_img, img_to_array
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import pathlib
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
from keras.utils.vis_utils import plot_model

from PIL import Image
import numpy as np
from skimage import transform
from os import walk

sys.path.append('/content/drive/MyDrive/Master/1.Sem/EmbeddedSystems/Final_Project/')

In [ ]:
train_path = "/content/drive/MyDrive/Master/1.Sem/EmbeddedSystems/Final_Project/Dataset_25by25/Training/"
test_path = "/content/drive/MyDrive/Master/1.Sem/EmbeddedSystems/Final_Project/Dataset_25by25/Test/"
Figure_path = "/content/drive/MyDrive/Master/1.Sem/EmbeddedSystems/Final_Project/PlotAndfigure/"
number_of_classes = 2

In [ ]:
NUMBER_OF_CLASSES = 2
FAST_RUN = False
IMAGE_WIDTH=25
IMAGE_HEIGHT=25
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS = 1

In [ ]:
filenames = os.listdir("/content/drive/MyDrive/Master/1.Sem/EmbeddedSystems/Final_Project/Dataset_25by25/Training")
categories = []
for filename in filenames:
    category = filename.split('_')[0]
    #print("category ", category)
    if category == 'pos0':
        categories.append(0)
    elif category == 'pos1':
        categories.append(1)
    elif category == 'pos2':
        categories.append(2)
    elif category == 'pos4':
        categories.append(3)
    else:
        categories.append(4)

df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})



In [ ]:
for index, row in df.iterrows():
  print( row['filename'], row['category'])
 

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
fig = plt.figure()
plt_bar = df['category'].value_counts().plot.bar()
plt.savefig('/content/drive/MyDrive/Master/1.Sem/EmbeddedSystems/Final_Project/PlotAndfigure/DataDistribution.pdf', bbox_inches='tight')
plt.savefig('/content/drive/MyDrive/Master/1.Sem/EmbeddedSystems/Final_Project/PlotAndfigure/DataDistribution.png', bbox_inches='tight')

# Visual images


In [ ]:
sample = random.choice(filenames)
image = load_img(train_path+sample)
plt.imshow(image)


# Build the CNN


In [ ]:
model = Sequential([
		Flatten(input_shape=(IMAGE_WIDTH,IMAGE_HEIGHT,IMAGE_CHANNELS)),		# reshape 10x10 to 100, layer 0
		Dense(64, activation='relu', use_bias=True),
		Dense(32, activation='relu', use_bias=True),
		Dense(NUMBER_OF_CLASSES, activation='softmax', use_bias=True),
	])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

In [ ]:
earlystop = EarlyStopping(patience=10)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:
callbacks = [earlystop, learning_rate_reduction]

In [ ]:
plot_model(model, to_file= Figure_path +'model_plot.pdf', show_shapes=True, show_layer_names=True)

# Preper data


In [ ]:
df["category"] = df["category"].replace({0: 'pos0', 1: 'pos1'}) #, 2: 'pos3', 3: 'pos4'


In [ ]:
train_df, validate_df = train_test_split(df, test_size=0.20, random_state=42)
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

In [ ]:
train_df['category'].value_counts().plot.bar()

plt.savefig(Figure_path + 'DataDistributionTraining.pdf', bbox_inches='tight')
plt.savefig(Figure_path +'DataDistributionTraining.png', bbox_inches='tight')


In [ ]:
validate_df['category'].value_counts().plot.bar()
plt.savefig(Figure_path + 'DataDistributionValidation.pdf', bbox_inches='tight')
plt.savefig(Figure_path +'DataDistributionValidation.png', bbox_inches='tight')


In [ ]:
total_train = train_df.shape[0]
total_validate = validate_df.shape[0]
batch_size= 64

In [ ]:
train_datagen = ImageDataGenerator(
    #rescale=1./255,
    #shear_range=0.1,
    #zoom_range=0.2,
    #horizontal_flip=True,
    #width_shift_range=0.1,
    #height_shift_range=0.1
)

train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    train_path, 
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    grayscale=True, 
    color_mode="grayscale"


)

In [ ]:
validation_datagen = ImageDataGenerator()#rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    train_path, 
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    grayscale=True, color_mode="grayscale"
)

In [ ]:
epochs=3 if FAST_RUN else 100
history = model.fit(train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size,
    callbacks=callbacks
)

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.savefig(Figure_path + 'TrainingAccuracy.pdf', bbox_inches='tight')
plt.savefig(Figure_path +'TrainingAccuracy.png', bbox_inches='tight')

plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.savefig(Figure_path + 'TrainingLoss.pdf', bbox_inches='tight')
plt.savefig(Figure_path +'TrainingLoss.png', bbox_inches='tight')
plt.show()

#Predict single image


In [ ]:


image = load("/content/drive/MyDrive/Master/1.Sem/EmbeddedSystems/Final_Project/Dataset_merged_faces_no_faces/Merged_Test/pos1_2838.jpg",)
pred = model.predict(image)
print("result ", pred)

index_max = np.argmax(pred,axis=1)
print(" Predicted pos ", index_max)


# Predict Test Dataset

In [ ]:
# Predict image
def load(filename):
   np_image = Image.open(filename)
   np_image = np.array(np_image).astype('float32')
   np_image = transform.resize(np_image, (25, 25, 1))
   np_image = np.expand_dims(np_image, axis=0)
   
   return np_image

In [ ]:
Test_names = []
test_images = test_path
for (dirpath, dirnames, filenames) in walk(test_images):
    Test_names.extend(filenames)


In [ ]:
y_gt = []
y_predict = []

for test in Test_names:
  image = load(test_images + test)
  pred = model.predict(image)
  index_max = np.argmax(pred,axis=1)
  temp = test.split('_', 1)[0]
  gt =  temp[len(temp)-1]
  int_gt = int(gt)

  y_predict.append(index_max)
  y_gt.append(int_gt)
  


In [ ]:
from sklearn.metrics import confusion_matrix

result = confusion_matrix( y_predict , y_gt, normalize='pred')
print(result)

In [ ]:
plt.figure(figsize = (15,15))

x_axis_labels = ["pos0","pos1"] # labels for x-axis

sns.heatmap(pd.DataFrame(result), xticklabels = x_axis_labels, yticklabels = x_axis_labels ,  annot = True, cmap="Blues")

plt.savefig(Figure_path + 'TestConfusionMatrix.pdf', bbox_inches='tight')
plt.savefig(Figure_path +'TestConfusionMatrix.png', bbox_inches='tight')



#Export the weight


In [ ]:
for w in range(1, len(model.layers)):
  weight_filename = "/content/drive/MyDrive/Master/1.Sem/EmbeddedSystems/Final_Project/Weights_25by25/layer_" + str(w) + "_weights.txt" 
  open(weight_filename, 'w').close() # clear file
  file = open(weight_filename,"a") 
  file.write('{')
  for i in range(model.layers[w].weights[0].numpy().shape[0]):
    file.write('{')
    for j in range(model.layers[w].weights[0].numpy().shape[1]):
      file.write(str(model.layers[w].weights[0].numpy()[i][j]))
      if j != model.layers[w].weights[0].numpy().shape[1]-1:
        file.write(', ')
    file.write('}')
    if i != model.layers[w].weights[0].numpy().shape[0]-1:
      file.write(', \n')
  file.write('}')
  file.close()

network_weights = model.layers[1].weights
#print(network_weights)
layer_1_W = network_weights[0].numpy()

#Save image to txt file

In [ ]:
import cv2
import numpy as np

# read the image
image = cv2.imread("/content/drive/MyDrive/Master/1.Sem/EmbeddedSystems/Final_Project/Dataset_25by25/Test/pos0_1242.jpg", cv2.IMREAD_GRAYSCALE)


# convert the data type of the image to float
image = image.astype(float)


# normalize the image between 0 and 1
normalized_image = image #/ 255


In [ ]:
filename = "/content/drive/MyDrive/Master/1.Sem/EmbeddedSystems/Final_Project/Images_RawData/pos0_1242.txt"

#w tells python we are opening the file to write into it
outfile = open(filename, 'w')


for i in range(len(normalized_image)):
  for j in range(len(normalized_image[i])):
    g = float("{:.5f}".format(normalized_image[i][j]))
    print(g)
    nCounter = nCounter +1
    outfile.write(str(g)+ ',')